# Análise de Resultados de Benchmark

Este notebook analisa todos os CSVs de resultados e gera um resumo estatístico.

## Objetivo
Calcular estatísticas (média, desvio padrão, min, max) agrupadas por:
- Sistema (gRPC ou RabbitMQ)
- Operação (list, upload, download)
- Tamanho do arquivo (KB)
- Número de clientes


In [ ]:
import csv
import os
import statistics
from pathlib import Path
from collections import defaultdict
import pandas as pd


## Configuração


In [ ]:
# Configuração de diretórios
project_root = Path('..')
results_dir = project_root / 'results'
output_file = results_dir / 'results_summary.csv'

print(f"📁 Diretório de resultados: {results_dir.absolute()}")
print(f"📄 Arquivo de saída: {output_file.absolute()}")


## Leitura dos Arquivos CSV


In [ ]:
def read_csv_files(results_dir):
    """Lê todos os arquivos CSV de resultados."""
    all_results = []
    results_path = Path(results_dir)
    
    if not results_path.exists():
        print(f"❌ Diretório {results_dir} não encontrado!")
        return []
    
    csv_files = list(results_path.glob("benchmark_*.csv"))
    
    if not csv_files:
        print(f"⚠️  Nenhum arquivo CSV encontrado em {results_dir}")
        return []
    
    print(f"📊 Encontrados {len(csv_files)} arquivo(s) CSV")
    
    for csv_file in csv_files:
        try:
            with open(csv_file, 'r', encoding='utf-8') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    # Ignora linhas vazias ou inválidas
                    if not row.get('timestamp') or not row.get('success'):
                        continue
                    
                    # Apenas operações bem-sucedidas
                    if row.get('success', '').lower() != 'true':
                        continue
                    
                    all_results.append({
                        'system': row.get('system', ''),
                        'operation': row.get('operation', ''),
                        'file_size_kb': int(row.get('file_size_kb', 0)),
                        'clients': int(row.get('clients', 0)),
                        'rtt_ms': float(row.get('rtt_ms', 0))
                    })
        except Exception as e:
            print(f"⚠️  Erro ao ler {csv_file.name}: {e}")
            continue
    
    print(f"✅ Total de {len(all_results)} resultados válidos processados")
    return all_results

# Lê todos os CSVs
results = read_csv_files(results_dir)


## Cálculo de Estatísticas


In [ ]:
def calculate_statistics(results):
    """Calcula estatísticas agrupadas por sistema, operação, tamanho e clientes."""
    # Agrupa resultados por chave única
    grouped = defaultdict(list)
    
    for result in results:
        key = (
            result['system'],
            result['operation'],
            result['file_size_kb'],
            result['clients']
        )
        grouped[key].append(result['rtt_ms'])
    
    # Calcula estatísticas para cada grupo
    summary = []
    
    for (system, operation, file_size_kb, clients), rtt_values in grouped.items():
        if len(rtt_values) < 2:
            # Precisa de pelo menos 2 valores para desvio padrão
            stddev = 0.0
        else:
            stddev = statistics.stdev(rtt_values)
        
        summary.append({
            'system': system,
            'operation': operation,
            'file_size_kb': file_size_kb,
            'clients': clients,
            'mean_ms': statistics.mean(rtt_values),
            'stddev_ms': stddev,
            'min_ms': min(rtt_values),
            'max_ms': max(rtt_values),
            'count': len(rtt_values)
        })
    
    # Ordena por sistema, operação, tamanho, clientes
    summary.sort(key=lambda x: (x['system'], x['operation'], x['file_size_kb'], x['clients']))
    
    return summary

# Calcula estatísticas
print("📈 Calculando estatísticas...")
summary = calculate_statistics(results)
print(f"✅ {len(summary)} combinações únicas calculadas")


## Visualização dos Dados


In [ ]:
# Converte para DataFrame para visualização
df_summary = pd.DataFrame(summary)

# Mostra primeiras linhas
print("📊 Primeiras 10 linhas do resumo:")
display(df_summary.head(10))


In [ ]:
# Estatísticas gerais
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print("  Estatísticas Gerais")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

systems = df_summary['system'].unique()
operations = df_summary['operation'].unique()

print(f"Sistemas: {', '.join(sorted(systems))}")
print(f"Operações: {', '.join(sorted(operations))}")
print(f"Total de combinações: {len(df_summary)}")
print(f"Total de resultados processados: {df_summary['count'].sum():,}")


In [ ]:
# Resumo por sistema
print("\n📊 Resumo por Sistema:")
display(df_summary.groupby('system').agg({
    'mean_ms': 'mean',
    'count': 'sum'
}).round(3))


In [ ]:
# Resumo por operação
print("\n📊 Resumo por Operação:")
display(df_summary.groupby('operation').agg({
    'mean_ms': 'mean',
    'count': 'sum'
}).round(3))


## Salvando o CSV de Resumo


In [ ]:
def write_summary_csv(summary, output_file):
    """Escreve o CSV de resumo."""
    fieldnames = ['system', 'operation', 'file_size_kb', 'clients', 
                  'mean_ms', 'stddev_ms', 'min_ms', 'max_ms']
    
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        
        for row in summary:
            writer.writerow({
                'system': row['system'],
                'operation': row['operation'],
                'file_size_kb': row['file_size_kb'],
                'clients': row['clients'],
                'mean_ms': f"{row['mean_ms']:.3f}",
                'stddev_ms': f"{row['stddev_ms']:.3f}",
                'min_ms': f"{row['min_ms']:.3f}",
                'max_ms': f"{row['max_ms']:.3f}"
            })
    
    print(f"✅ Resumo salvo em: {output_file}")
    print(f"📊 Total de {len(summary)} combinações únicas")

# Salva o CSV
write_summary_csv(summary, output_file)
